In [1]:
%load_ext autoreload
%autoreload 2

# Simple IEX Stock Fetcher

The IEX free message quota only gets one year of data for 200 stocks, so run this, fetcher stock data and save to disk every month.


## Test api

In [2]:
import iex
import key
iex.init(key.test_token, api='sandbox')
aapl = iex.Stock('AAPL')
aapl.chart('5d', chartCloseOnly=True)

,change,changeOverTime,changePercent,close,volume
date,,,,,
2019-10-01,0.630000,0.002846,0.2776,235.08,36626895
2019-10-02,-5.740000,-0.023081,-2.5109,222.51,35941685
2019-10-03,1.940000,-0.014185,0.8914,231.78,30438546
2019-10-04,6.310000,0.013995,2.8347,230.24,34995248
2019-10-07,0.051249,0.000000,0.0200,234.46,32130948


In [3]:
aapl.dividends('1y')

,amount,currency,date,declaredDate,description,flag,frequency,paymentDate,recordDate
exDate,,,,,,,,,
2018-11-14,0.74,USD,2019-10-08,2018-11-10,er eD lrpfpl$esAa7reP a udDiv. h3 areSeeydicr ...,QhneQoNCao g,reayurltQ,2018-11-29,2018-11-14


## Fetcher SPY stocks

In [4]:
import pandas as pd
spy = [x for x in pd.read_html('https://etfdailynews.com/etf/spy/', attrs={'id': 'etfs-that-own'})[0].Symbol.values.tolist() 
       if isinstance(x, str)]
correction = {'PCLN':'BKNG', 'DWDP':'DD'}

In [5]:
import os, sys
chart_range = [(5,'5d'), (20,'1m'), (75,'3m'), (165,'6m'), (341,'1y'), (715,'2y'), (99999,'max')]

In [ ]:
# iex.init(key.test_token, api='sandbox')
iex.init(key.token, api='cloud')

# save historical price to disk
for k in spy:
    k = correction[k] if k in correction else k
    sys.stdout.write('\r{}   '.format(k))
    filename = "./daily/{}.csv".format(k)
    if os.path.exists(filename):
        # fetcher missing period
        df = pd.read_csv(filename, index_col='date', parse_dates=True)
        days = pd.Timedelta(df.index[-1] - pd.datetime.now()).days
        bdays = len(pd.bdate_range(df.index[-1], pd.datetime.now())) - 1
        for day, param in chart_range:
            if bdays <= 0:
                break
            elif days < day:
                df_append = iex.Stock(k).chart(param)
                pd.concat([df, df_append]).to_csv(filename)
                break
    else:
        iex.Stock(k).chart('max').to_csv(filename)
    
